In [ ]:
# # 날짜 간 차이 계산
# df['next_date'] = df['date'] + pd.Timedelta(days=1)
# df['date_diff'] = df['next_date'] - df['date']

# print(df)

# # 시간대 설정 및 변환
# df['date_utc'] = df['date'].dt.tz_localize('UTC')
# df['date_kst'] = df['date_utc'].dt.tz_convert('Asia/Seoul')


# # 범주형 피처 인코딩
# df = pd.get_dummies(df, columns=['destination'], drop_first=True)

# # 피처와 타겟 분리
# X = df.drop(columns=['price'])
# y = df['price']

# # 비행시간 피처 포함 모델
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# model = RandomForestRegressor(n_estimators=100, random_state=42)
# model.fit(X_train, y_train)
# y_pred = model.predict(X_test)
# print("MSE with flight_duration:", mean_squared_error(y_test, y_pred))

# # 비행시간 피처 제거 모델
# X = X.drop(columns=['flight_duration'])
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# model = RandomForestRegressor(n_estimators=100, random_state=42)
# model.fit(X_train, y_train)
# y_pred = model.predict(X_test)
# print("MSE without flight_duration:", mean_squared_error(y_test, y_pred))


In [ ]:
!git clone --recursive https://github.com/microsoft/LightGBM
%cd LightGBM
!cmake -B build -S .
!cmake --build build -j4

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, accuracy_score

import pandas as pd
import numpy as np

import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

# 출발일로 부터 항공권 예매 시점(20240430)과의 날짜 차이 피처 추가 예정
now = pd.to_datetime("20240430", format='%Y%m%d')
print("Timestamp:", now)

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    df = pd.read_csv('/content/drive/MyDrive/직항_머신러닝.csv')
except:
    df = pd.read_csv('./data/직항_머신러닝.csv')

In [ ]:
df.describe(include='all')

In [ ]:
df.info()

In [ ]:
# jt 비행시간 350은 03시간50분으로 230분으로 변환
df['departure_jt'] = df['departure_jt'].apply(lambda x : str(x).zfill(4)).apply(lambda x : int(x[:2])*60+int(x[2:]))
df['arrival_jt'] = df['arrival_jt'].apply(lambda x : str(x).zfill(4)).apply(lambda x : int(x[:2])*60+int(x[2:]))

df[['departure_jt','arrival_jt']].head()

In [ ]:
# str to datetime 형식 변환
df['departure_sdt'] = pd.to_datetime(df['departure_sdt'])
df['arrival_sdt'] = pd.to_datetime(df['arrival_sdt'])

# 시간 분 추출
df['departure_hour'] = df['departure_sdt'].apply(lambda x: x.hour)
df['departure_minute'] = df['departure_sdt'].apply(lambda x: x.minute)
df['arrival_hour'] = df['arrival_sdt'].apply(lambda x: x.hour)
df['arrival_minute'] = df['arrival_sdt'].apply(lambda x: x.minute)

# 출발 도착 시 분 sin cos 변환
df['departure_hour_sin'] = np.sin(2 * np.pi * df['departure_hour'] / 24)
df['departure_hour_cos'] = np.cos(2 * np.pi * df['departure_hour'] / 24)
df['departure_minute_sin'] = np.sin(2 * np.pi * df['departure_minute'] / 60)
df['departure_minute_cos'] = np.cos(2 * np.pi * df['departure_minute'] / 60)

df['arrival_hour_sin'] = np.sin(2 * np.pi * df['arrival_hour'] / 24)
df['arrival_hour_cos'] = np.cos(2 * np.pi * df['arrival_hour'] / 24)
df['arrival_minute_sin'] = np.sin(2 * np.pi * df['arrival_minute'] / 60)
df['arrival_minute_cos'] = np.cos(2 * np.pi * df['arrival_minute'] / 60)

# 출발 도착 요일 sin cos 변환
df['dep_week_sin'] = np.sin(2 * np.pi * df['dep_week'] / 7)
df['dep_week_cos'] = np.cos(2 * np.pi * df['dep_week'] / 7)
df['arr_week_sin'] = np.sin(2 * np.pi * df['arr_week'] / 7)
df['arr_week_cos'] = np.cos(2 * np.pi * df['arr_week'] / 7)


drop_col = ['departure_sdt','arrival_sdt',
            'departure_hour','departure_minute','arrival_hour','arrival_minute',
            'dep_week', 'arr_week']
df.drop(columns=drop_col,inplace=True)

In [ ]:
# departure_jt, arrival_jt 출발 복귀 비행시간
# 직항 항공권 기준 목적지 피처와 다중공산성 문제가 있다고 예상 drop 전후로 모델 학습 후 판단.
# 경유 항공권 또는 비행기 도착시 현지시간으로 변환 할 때 사용

# drop_jt = ['departure_jt', 'arrival_jt']
# df.drop(columns=drop_jt,inplace=True)

In [ ]:

# agentcode
# 대행사 별 주력 여행지가 있는지 시각화 필요
# 대항사와 여행지의 상관관계가 크다면 다중공산성 의심
# 대행사와 항공권 가격의 상관관계 확인 필요
df['agentcode'].value_counts()

In [ ]:
# baggagetype
# "P" 수화물 포함
# [F, C] 수화물 미포함
df['baggagetype'].value_counts()

In [ ]:
# 'C', Nan 값을 F로 치환
df['baggagetype'] = df['baggagetype'].apply(lambda x: 'F' if x == 'C' else x).fillna('F')
df['baggagetype'].value_counts(normalize=True)

In [ ]:
df.info()

In [ ]:
df

In [ ]:
def optimize_dataframe(df):
    """
    데이터프레임의 dtype을 최소화하여 메모리를 최적화하는 함수.

    Parameters:
    df (pd.DataFrame): 최적화할 데이터프레임

    Returns:
    pd.DataFrame: 최적화된 데이터프레임
    """
    # 정수형 컬럼 최적화
    int_columns = df.select_dtypes(include=['int64']).columns
    for col in int_columns:
        if df[col].min() >= np.iinfo(np.int8).min and df[col].max() <= np.iinfo(np.int8).max:
            df[col] = df[col].astype(np.int8)
        elif df[col].min() >= np.iinfo(np.int16).min and df[col].max() <= np.iinfo(np.int16).max:
            df[col] = df[col].astype(np.int16)
        elif df[col].min() >= np.iinfo(np.int32).min and df[col].max() <= np.iinfo(np.int32).max:
            df[col] = df[col].astype(np.int32)
        else:
            df[col] = df[col].astype(np.int64)

    # 부동소수점 컬럼 최적화
    float_columns = df.select_dtypes(include=['float64']).columns
    for col in float_columns:
        df[col] = df[col].astype(np.float32)

    # 객체형 컬럼 최적화
    object_columns = df.select_dtypes(include=['object']).columns
    for col in object_columns:
        num_unique_values = len(df[col].unique())
        num_total_values = len(df[col])
        if num_unique_values / num_total_values < 0.5:
            df[col] = df[col].astype('category')

    return df
df = optimize_dataframe(df)

In [ ]:
# 메모리 최적화로 메모리 1/3으로 감소
df.info()

In [ ]:
# 사용하지 않을 컬럼 분리
# tz = df['time_zone']
# df = df.drop(columns=['time_zone'])

# 피처와 타겟 변수 분리
X = df.drop(columns=['total_fare'])
y = df['total_fare']


# 범주형 피처 식별
categorical_features = ['agentcode', 'partition_0', 'partition_1', 'time_zone', 'baggagetype']

# 라벨 인코딩 적용
label_encoders = {}
for col in categorical_features:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le

In [ ]:
# 학습 데이터, 검증 데이터, 테스트 데이터 분리
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# LightGBM 데이터셋 생성
train_data = lgb.Dataset(X_train, label=y_train)
val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)


In [ ]:
# import seaborn as sns

# plt.figure(figsize=(10, 8))
# sns.heatmap(df.select_dtypes(include=[int, float]).corr(), annot=True, cmap='coolwarm', linewidths=0.5)
# plt.title('Correlation Matrix')
# plt.show()

In [ ]:
params = {
    'objective': 'regression',  # 회귀
    'metric': 'rmse',  # 평가 지표 (Root Mean Squared Error)
    'boosting_type': 'gbdt',
    'learning_rate': 0.1,
    'num_leaves': 31,
    'max_depth': -1,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1,
    'lambda_l2': 1.0,  # L2 정규화 (과적합 방지)
    'device': 'gpu'  # GPU 사용 설정
}

In [ ]:
from tqdm import tqdm

# 학습 진행도를 출력하기 위해 콜백 함수 사용
tqdm_callback = lgb.callback.log_evaluation(period=50)
early_stopping_callback = lgb.early_stopping(stopping_rounds=50)

# LightGBM 모델 학습
model = lgb.train(params, train_data, valid_sets=[val_data], num_boost_round=10000, callbacks=[tqdm_callback, early_stopping_callback])

In [ ]:
# 테스트 데이터에 대한 예측
y_pred = model.predict(X_test, num_iteration=model.best_iteration)

# 모델 평가
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
print(f'RMSE: {rmse:.4f}')

In [ ]:
import matplotlib.pyplot as plt

# 피처 중요도 추출
importance = model.feature_importance(importance_type='split')
feature_names = model.feature_name()

importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importance
})

# 중요도 순으로 정렬
importance_df = importance_df.sort_values(by='importance', ascending=False)

# 피처 중요도 시각화
plt.figure(figsize=(10, 6))
plt.barh(importance_df['feature'], importance_df['importance'])
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Feature Importance')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import matplotlib.pyplot as plt
import seaborn as sns
def analyze_multicollinearity(X, y, figsize=(14, 8)):

    # 다중공산성 계산
    vif = pd.DataFrame()
    vif['Feature'] = X.columns
    vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    print(vif)

    # 상관 행렬 계산 및 시각화
    corr_data = pd.concat([X, y], axis=1).corr().applymap(lambda x: round(x, 2))
    plt.figure(figsize=figsize)
    sns.heatmap(corr_data, annot=True, cmap='coolwarm', linewidths=0.5)
    plt.title('Correlation Matrix')
    plt.show()

    # 상수항 추가 (OLS 회귀를 위해)
    X = sm.add_constant(X)

    # OLS 회귀 모델 적합
    model = sm.OLS(y, X).fit()

    # 모델 요약 출력
    print(model.summary())


drop_col = ['partition_0', 'baggagetype', 'departure_jt', 'arr_week_sin','partition_1', 'time_zone',
            'departure_hour_sin']
droped_X = X.drop(columns = drop_col)

analyze_multicollinearity(droped_X,y)

In [ ]:
# 학습 데이터, 검증 데이터, 테스트 데이터 분리
X_train, X_temp, y_train, y_temp = train_test_split(droped_X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# LightGBM 데이터셋 생성
train_data = lgb.Dataset(X_train, label=y_train)
val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

model = lgb.train(params, train_data, valid_sets=[val_data], num_boost_round=10000, callbacks=[tqdm_callback, early_stopping_callback])

In [ ]:
# 테스트 데이터에 대한 예측
y_pred = model.predict(X_test, num_iteration=model.best_iteration)

# 모델 평가
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
print(f'RMSE: {rmse:.4f}')

In [ ]:
# 피처 중요도 추출
importance = model.feature_importance(importance_type='split')
feature_names = model.feature_name()

importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importance
})

# 중요도 순으로 정렬
importance_df = importance_df.sort_values(by='importance', ascending=False)

# 피처 중요도 시각화
plt.figure(figsize=(10, 6))
plt.barh(importance_df['feature'], importance_df['importance'])
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Feature Importance')
plt.gca().invert_yaxis()
plt.show()